In [221]:
import pandas as pd
from datamatch import (
    JaroWinklerSimilarity,
    ThresholdMatcher,
    ColumnsIndex,
)

In [222]:
def read_data():
    df = pd.read_csv("../data/OK/aggregate_members_oathkeepers_cleaned.csv")
    return df 

In [223]:
dfa = read_data()

In [224]:
dfa = dfa.fillna("")

In [225]:
dfa.loc[:, "state"] = dfa.state.str.lower().str.strip().str.replace(r"ma\.", "ma", regex=True)

In [226]:
dfa.state.unique()

array(['wy', 'me', 'nm', 'nh', 'ok', 'nc', 'nj', 'ca', 'ga', 'vt', 'ak',
       'id', 'ri', 'tx', 'pa', 'ne', 'ny', 'in', 'mt', 'or', 'wa', 'ma',
       'al', 'va', 'il', 'nv', 'wi', 'fl', 'mi', 'oh', 'co', 'ct', 'sc',
       'la', 'ia', 'ar', 'az', 'tn', 'ut', 'ks', 'ky', 'mn', 'wa.', 'mo',
       'md', 'hi', 'wv', 'ms', 'dc', 'de', 'sd', 'nd', 'ae', '', 'canada',
       'grand island', 'sacramento', 'thailand', 'ne.', 'fl  (az)', 'la.',
       'n.y.', 'pa.', 'iowa', 'bc canada', '??', 'austraila pto', 'ap',
       'lv', 'jacksonville', 'bc', 'oslo', 'alberta, canada',
       'western australia', 'rl', 'carolina beach', '--', 'cn', 'pr',
       'md�', 'florida', '9000', 'stewartstown', 'roosevelt',
       'ca  nonono', 'e7n2v2', 'st croix', 'hamden', 'ontario',
       '607ws/dow, unit: 15173, apo, ap 96205, united states',
       'oh  start', 'mo  end', 'ny11580', '8724', 'ny11766', 'nc28773',
       'sterling heights'], dtype=object)

In [227]:
dfa = dfa[dfa.state.isin(["ma"])]
dfa.loc[:, "notes"] = dfa.notes.str.lower().str.strip()
dfa.loc[:, "first_name"] = dfa.first_name.str.replace(r"^ (\w+)", r"\1", regex=True).str.replace(r"(\w+) $", r"\1", regex=True)
dfa.loc[:, "last_name"] = dfa.last_name.str.replace(r"^ (\w+)", r"\1", regex=True).str.replace(r"(\w+) $", r"\1", regex=True)
dfa.loc[:, "city"] = dfa.city.str.lower().str.strip().str.replace(r"\,.+", "", regex=True)

In [228]:
search_1 = dfa[dfa.notes.str.contains("law enforcement")]
search_1

,id_number,group_name,bool,city,state,email,undefined_date,notes,first_name,middle_name,last_name


In [229]:
search_2 = dfa[dfa.notes.str.contains("police")]
search_2

,id_number,group_name,bool,city,state,email,undefined_date,notes,first_name,middle_name,last_name
4122,1043,Annual,No,hopkinton,ma,cjdcarving@gmail.com,,"civilian - brothers are ret. mil, policeman, m...",cecilia,del,gaudio
8197,6075,Annual,No,westport,ma,David.Bellavance@us.ngrid.com,,cpr/fa/aed instuctor if people want to be cert...,david,,bellavance
13353,11462,Annual,No,braintree,ma,beungood8@aol.com,1/6/2012,i have been a fulltime sworn police officer f...,john,,ouellette


In [230]:
search_3 = dfa[dfa.notes.str.contains("patrol")]
search_3.notes.unique()

array(['i am a sergeant supervising a patrol shift in falmouth, massachusetts.  i am a member of the sandwich american legion post 188 and a member of the leathernecks motorcycle club.'],
      dtype=object)

In [231]:
search_4 = dfa[dfa.notes.str.contains("officer")]
search_4.notes.unique()

array(["also worked as k9 officer, also worked with nd attorney general's drug enforcement unit",
       "cpr/fa/aed instuctor if people want to be certified i can provide that. first responder trained for medical if needed. the skills i've learned as a soldier and part time police officer.",
       'i have been  a fulltime sworn police officer for 16 years.  i am into cycling,hunting fishing,martial arts ,kayaking and cross country skiing'],
      dtype=object)

In [232]:
search_5 = dfa[dfa.notes.str.contains("swat")]
search_5

,id_number,group_name,bool,city,state,email,undefined_date,notes,first_name,middle_name,last_name


In [233]:
search_6 = dfa[dfa.notes.str.contains("explosive")]
search_6

,id_number,group_name,bool,city,state,email,undefined_date,notes,first_name,middle_name,last_name


In [234]:
search_7 = dfa[dfa.notes.str.contains("bomb")]
search_7

,id_number,group_name,bool,city,state,email,undefined_date,notes,first_name,middle_name,last_name


In [235]:
search_8 = dfa[dfa.notes.str.contains("active duty")]
search_8.notes.unique()

array(['spreading the word, and god forbid forming a neighborhood / town militia to keep the oath. i am the son of a us army msgt who died active duty (cid)1971, british boys brigade at 8yo,shot my first m60 at age of 9 at green berit day new cumberland army depot,attended west point youth camp 4 years.i know what kind of people can be counted on and trusted.and i will never submit to any enemy foriegn or domestic.'],
      dtype=object)

In [236]:
dfs = [search_1, search_2, search_3, search_4, search_5, search_6, search_7, search_8]
dfb = pd.concat(dfs, axis=0)

In [237]:
def read_misconduct_data():
    df = pd.read_csv("../data/MA/pprr_ma.csv")
    return df

In [238]:
dfc = read_misconduct_data()
dfc = dfc.rename(columns={"Full_Name": "full_name", "Agency": "city", "County": "county"})
dfc.loc[:, "city"] = dfc.city.str.lower().str.strip().str.replace(r" police.+", "", regex=True)
def split_names(df):
    names = df.full_name.str.lower().str.strip().str.extract(r"^(\w+) (.+)")
    df.loc[:, "first_name"] = names[0]
    df.loc[:, "last_name"] = names[1]
    return df

In [239]:
dfc = dfc.pipe(split_names)

In [240]:

def match_ok_names_to_personnel(ok, agency):
    dfa = ok[["first_name", "last_name", "notes", "city"]]
    dfa["county"] = ""
    dfa.loc[:, "fc"] = dfa.first_name.fillna("").map(lambda x: x[:1])
    dfa.loc[:, "lc"] = dfa.last_name.fillna("").map(lambda x: x[:1])
    dfa = dfa.drop_duplicates(subset=["first_name", "last_name"])

    dfb = agency[["first_name", "last_name", "city", "county"]]
    dfb["notes"] = ""
    dfb.loc[:, "fc"] = dfb.first_name.fillna("").map(lambda x: x[:1])
    dfb.loc[:, "lc"] = dfb.last_name.fillna("").map(lambda x: x[:1])
    dfb = dfb.drop_duplicates(subset=["first_name", "last_name"])

    matcher = ThresholdMatcher(
        ColumnsIndex(["fc", "lc"]),
        {
            "first_name": JaroWinklerSimilarity(),
            "last_name": JaroWinklerSimilarity(),
        },
        dfa,
        dfb,
    )
    decision = 0.800
    matcher.save_pairs_to_excel(
        ("../data/MA/oathkeeper_names_v_MA_full_roster.xlsx"), decision
    )
    return matcher

In [241]:
match = match_ok_names_to_personnel(dfa, dfc)

C:\Users\PC\AppData\Local\Temp\ipykernel_7720\2252717086.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa["county"] = ""
C:\Users\PC\AppData\Local\Temp\ipykernel_7720\2252717086.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa.loc[:, "fc"] = dfa.first_name.fillna("").map(lambda x: x[:1])
C:\Users\PC\AppData\Local\Temp\ipykernel_7720\2252717086.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [ ]:
# def read_matches():
#     df = pd.read_excel("../data/")
#     return df 

In [ ]:
# matches = read_matches()

In [ ]:
# matches